<!--NOTEBOOK_HEADER-->
This notebook contain AI based energy transition examples from [AI energy transition group](https://github.com/ShengrenHou/Energy-Data-and-Model-Platform);
content is available [on Github](https://github.com/ShengrenHou/Energy-Data-and-Model-Platform).

<!--NAVIGATION-->
1.1 60 Minutes to Deep reinforcement learning based Energy Management: An mock up energy system example

<p><a href="https://colab.research.google.com/github/ShengrenHou/Energy-Data-and-Model-Platform/blob/main/Examples/Example_for_forecast_HSR_revise.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://github.com/ShengrenHou/Energy-Data-and-Model-Platform/blob/main/Examples/Example_for_forecast_HSR_revise.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

### Energy Management 

Energy management is the process of monitoring, controlling, and conserving energy in a building or system. It involves the use of technology and systems to optimize energy consumption, reduce waste and lower costs. Effective energy management can result in significant savings on energy bills, reduce the environmental impact of energy consumption and improve overall energy efficiency. Energy management can be applied to a wide range of contexts, including residential, commercial, and industrial settings.

References:
-[Performance Comparison of Deep RL Algorithms for Energy Systems Optimal Scheduling](https://ieeexplore.ieee.org/document/9960642)

### Reinforcement Learning Algorithms

Reinforcement Learning (RL) is a subfield of machine learning where an agent learns to make decisions by interacting with an environment. The agent receives feedback in the form of rewards or punishments based on the actions it takes, and the goal of the agent is to maximize its cumulative reward over time.

In RL, the agent learns through trial and error, adjusting its behavior based on the feedback it receives from the environment. The agent uses a policy to map observations of the environment to actions, and it can either learn a new policy through exploration or improve an existing policy through exploitation. RL has been successfully applied to a variety of tasks, including game playing, robotics, and autonomous driving.

## Package preparation 


Install [PyTorch](https://pytorch.org/). 
The implemented DDPG algorithm is based on PyTorch package. 


In [ ]:
! pip3 install -U pytorch

In [ ]:
import os
import pickle
import gym
import time
import torch
import torch.nn as nn
import numpy as np
import numpy.random as rd
from torch.nn.modules import loss
from random_generator_battery import ESSEnv
import pandas as pd
import random
import gym
from gym import spaces  

## Data and Environment Prepartion 


### Data download and read 

Download data from our [github](https://github.com/ShengrenHou/Energy-Data-and-Model-Platform) link

In [ ]:
!git clone https://github.com/ShengrenHou/Energy-Data-and-Model-Platform

Read csv data as pandas dataframe and use the first column as the index (time index)

In [ ]:
pv_data=pd.read_csv('./Energy-Data-and-Model-Platform/Data/PV.csv')
h4_data=pd.read_csv('./Energy-Data-and-Model-Platform/Data/H4.csv')
price_data=pd.read_csv('./Energy-Data-and-Model-Platform/Data/Prices.csv')

### Define Environment

An environment for energy management typically includes a model of the energy system being managed, which may include various components such as generators, storage systems, loads, and renewable energy sources. The environment may also include a set of constraints and objectives that the energy management algorithm needs to satisfy, such as power balance constraints, economic cost minimization, or environmental impact minimization.

In a reinforcement learning context, the environment provides feedback to the learning agent in the form of rewards or penalties based on the actions it takes. The agent's goal is to learn a policy that maximizes the cumulative reward over time, while satisfying the constraints and objectives of the energy management problem.

The specific details of the environment will depend on the particular energy management problem being addressed, such as the type of energy system, the constraints and objectives, and the available data and sensors.
To define an environment for a energy management problem, you can start by considering the following components:
State Space: 

Action Space: 

Reward Function:
 
Dynamics: 

Define parameters for battery and DGs 

In [ ]:
battery_parameters={
'capacity':500,# kw
'max_charge':100, # kw
'max_discharge':100, #kw
'efficiency':0.9,
'degradation':0, #euro/kw
'max_soc':0.8,
'min_soc':0.2,
'initial_capacity':0.2}

dg_parameters={
'gen_1':{'a':0.0034
,'b': 3 
,'c':30
,'d': 0.03,'e':4.2,'f': 0.031,'power_output_max':150,'power_output_min':10,'heat_output_max':None,'heat_output_min':None,\
'ramping_up':100,'ramping_down':100,'min_up':2,'min_down':1},

'gen_2':{'a':0.001
,'b': 10
,'c': 40
,'d': 0.03,'e':4.2,'f': 0.031,'power_output_max':375,'power_output_min':50,'heat_output_max':None,'heat_output_min':None,\
    'ramping_up':100,'ramping_down':100,'min_up':2,'min_down':1},

'gen_3':{'a':0.001
,'b': 15
,'c': 70
,'d': 0.03,'e':4.2,'f': 0.031,'power_output_max':500,'power_output_min':100,'heat_output_max':None,'heat_output_min':None,\
    'ramping_up':200,'ramping_down':200,'min_up':2,'min_down':1}}

Define Environment 

In [ ]:
class Constant:
	MONTHS_LEN = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
	MAX_STEP_HOURS = 24 * 30
class DataManager():
    def __init__(self) -> None:
        self.PV_Generation=[]
        self.Prices=[]
        self.Electricity_Consumption=[]
    def add_pv_element(self,element):self.PV_Generation.append(element)
    def add_price_element(self,element):self.Prices.append(element)
    def add_electricity_element(self,element):self.Electricity_Consumption.append(element)

    # get current time data based on given month day, and day_time
    def get_pv_data(self,month,day,day_time):return self.PV_Generation[(sum(Constant.MONTHS_LEN[:month-1])+day-1)*24+day_time]
    def get_price_data(self,month,day,day_time):return self.Prices[(sum(Constant.MONTHS_LEN[:month-1])+day-1)*24+day_time]
    def get_electricity_cons_data(self,month,day,day_time):return self.Electricity_Consumption[(sum(Constant.MONTHS_LEN[:month-1])+day-1)*24+day_time]
    # get series data for one episode
    def get_series_pv_data(self,month,day): return self.PV_Generation[(sum(Constant.MONTHS_LEN[:month-1])+day-1)*24:(sum(Constant.MONTHS_LEN[:month-1])+day-1)*24+24]
    def get_series_price_data(self,month,day):return self.Prices[(sum(Constant.MONTHS_LEN[:month-1])+day-1)*24:(sum(Constant.MONTHS_LEN[:month-1])+day-1)*24+24]
    def get_series_electricity_cons_data(self,month,day):return self.Electricity_Consumption[(sum(Constant.MONTHS_LEN[:month-1])+day-1)*24:(sum(Constant.MONTHS_LEN[:month-1])+day-1)*24+24]

class DG():
    '''simulate a simple diesel generator here'''
    def __init__(self,parameters):
        self.name=parameters.keys()
        self.a_factor=parameters['a']
        self.b_factor=parameters['b']
        self.c_factor=parameters['c']
        self.power_output_max=parameters['power_output_max']
        self.power_output_min=parameters['power_output_min']
        self.ramping_up=parameters['ramping_up']
        self.ramping_down=parameters['ramping_down']
        self.last_step_output=None 
    def step(self,action_gen):
        output_change=action_gen*self.ramping_up# constrain the output_change with ramping up boundary
        output=self.current_output+output_change
        if output>0:
            output=max(self.power_output_min,min(self.power_output_max,output))# meet the constrain 
        else:
            output=0
        self.current_output=output
    def _get_cost(self,output):
        if output<=0:
            cost=0
        else:
            cost=(self.a_factor*pow(output,2)+self.b_factor*output+self.c_factor)
        return cost 
    def reset(self):
        self.current_output=0

class Battery():
    '''simulate a simple battery here'''
    def __init__(self,parameters):
        self.capacity=parameters['capacity']
        self.max_soc=parameters['max_soc']
        self.initial_capacity=parameters['initial_capacity']
        self.min_soc=parameters['min_soc']# 0.2
        self.degradation=parameters['degradation']# degradation cost 1.2
        self.max_charge=parameters['max_charge']# nax charge ability
        self.max_discharge=parameters['max_discharge']
        self.efficiency=parameters['efficiency']
    def step(self,action_battery):
        energy=action_battery*self.max_charge
        updated_capacity=max(self.min_soc,min(self.max_soc,(self.current_capacity*self.capacity+energy)/self.capacity))
        self.energy_change=(updated_capacity-self.current_capacity)*self.capacity# if charge, positive, if discharge, negative
        self.current_capacity=updated_capacity# update capacity to current codition
    def _get_cost(self,energy):# calculate the cost depends on the energy change
        cost=energy**2*self.degradation
        return cost  
    def SOC(self):
        return self.current_capacity
    def reset(self):
        self.current_capacity=np.random.uniform(0.2,0.8)
class Grid():
    def __init__(self):
        
        self.on=True
        if self.on:
            self.exchange_ability=100
        else:
            self.exchange_ability=0
    def _get_cost(self,current_price,energy_exchange):
        return current_price*energy_exchange
    def retrive_past_price(self):
        result=[]
        if self.day<1:
            past_price=self.past_price#
        else:
            past_price=self.price[24*(self.day-1):24*self.day]
            # print(past_price)
        for item in past_price[(self.time-24)::]:
            result.append(item)
        for item in self.price[24*self.day:(24*self.day+self.time)]:
            result.append(item)
        return result 
class ESSEnv(gym.Env):
    def __init__(self,**kwargs):
        super(ESSEnv,self).__init__()
        #parameters 
        self.data_manager=DataManager()
        self._load_year_data()
        self.episode_length=kwargs.get('episode_length',24)
        self.month=None
        self.day=None
        self.TRAIN=True
        self.current_time=None
        self.battery_parameters=kwargs.get('battery_parameters',battery_parameters)
        self.dg_parameters=kwargs.get('dg_parameters',dg_parameters)
        self.penalty_coefficient=50#control soft penalty constrain 
        self.sell_coefficient=0.5# control sell benefits

        self.grid=Grid()
        self.battery=Battery(self.battery_parameters)
        self.dg1=DG(self.dg_parameters['gen_1'])
        self.dg2=DG(self.dg_parameters['gen_2'])
        self.dg3=DG(self.dg_parameters['gen_3'])

        self.action_space=spaces.Box(low=-1,high=1,shape=(4,),dtype=np.float32)

        self.state_space=spaces.Box(low=0,high=1,shape=(7,),dtype=np.float32)

    @property
    def netload(self):

        return self.demand-self.grid.wp_gen-self.grid.pv_gen
        
    def reset(self,):
        self.month=np.random.randint(1,13)# here we choose 12 month
        if self.TRAIN:
            self.day=np.random.randint(1,20)
        else:
            self.day=np.random.randint(20,Constant.MONTHS_LEN[self.month]-1)
        self.current_time=0
        self.battery.reset()
        self.dg1.reset()
        self.dg2.reset()
        self.dg3.reset()
        return self._build_state()
    def _build_state(self):
        soc=self.battery.SOC()
        dg1_output=self.dg1.current_output
        dg2_output=self.dg2.current_output
        dg3_output=self.dg3.current_output
        time_step=self.current_time
        electricity_demand=self.data_manager.get_electricity_cons_data(self.month,self.day,self.current_time)
        pv_generation=self.data_manager.get_pv_data(self.month,self.day,self.current_time)
        price=self.data_manager.get_price_data(self.month,self.day,self.current_time)
        net_load=electricity_demand-pv_generation
        obs=np.concatenate((np.float32(time_step),np.float32(price),np.float32(soc),np.float32(net_load),np.float32(dg1_output),np.float32(dg2_output),np.float32(dg3_output)),axis=None)
        return obs

    def step(self,action):# state transition here current_obs--take_action--get reward-- get_finish--next_obs
        ## here we want to put take action into each components
        current_obs=self._build_state()
        self.battery.step(action[0])# here execute the state-transition part, battery.current_capacity also changed
        self.dg1.step(action[1])
        self.dg2.step(action[2])
        self.dg3.step(action[3])
        current_output=np.array((self.dg1.current_output,self.dg2.current_output,self.dg3.current_output,-self.battery.energy_change))#truely corresonding to the result
        self.current_output=current_output
        actual_production=sum(current_output)        
        netload=current_obs[3]
        price=current_obs[1]

        unbalance=actual_production-netload

        reward=0
        excess_penalty=0
        deficient_penalty=0
        sell_benefit=0
        buy_cost=0
        self.excess=0
        self.shedding=0
        if unbalance>=0:# it is now in excess condition
            if unbalance<=self.grid.exchange_ability:
                sell_benefit=self.grid._get_cost(price,unbalance)*self.sell_coefficient #sell money to grid is little [0.029,0.1]
            else:
                sell_benefit=self.grid._get_cost(price,self.grid.exchange_ability)*self.sell_coefficient
                #real unbalance that even grid could not meet 
                self.excess=unbalance-self.grid.exchange_ability
                excess_penalty=self.excess*self.penalty_coefficient
        else:# unbalance <0, its load shedding model, in this case, deficient penalty is used 
            if abs(unbalance)<=self.grid.exchange_ability:
                buy_cost=self.grid._get_cost(price,abs(unbalance))
            else:
                buy_cost=self.grid._get_cost(price,self.grid.exchange_ability)
                self.shedding=abs(unbalance)-self.grid.exchange_ability
                deficient_penalty=self.shedding*self.penalty_coefficient
        battery_cost=self.battery._get_cost(self.battery.energy_change)# we set it as 0 this time 
        dg1_cost=self.dg1._get_cost(self.dg1.current_output)
        dg2_cost=self.dg2._get_cost(self.dg2.current_output)
        dg3_cost=self.dg3._get_cost(self.dg3.current_output)

        reward-=(battery_cost+dg1_cost+dg2_cost+dg3_cost+excess_penalty+
        deficient_penalty-sell_benefit+buy_cost)/1e3
        self.operation_cost=battery_cost+dg1_cost+dg2_cost+dg3_cost+buy_cost-sell_benefit+excess_penalty+deficient_penalty
        self.unbalance=unbalance
        self.real_unbalance=self.shedding+self.excess
        final_step_outputs=[self.dg1.current_output,self.dg2.current_output,self.dg3.current_output,self.battery.current_capacity]
        self.current_time+=1
        finish=(self.current_time==self.episode_length)
        if finish:
            self.final_step_outputs=final_step_outputs
            self.current_time=0
            # self.day+=1
            # if self.day>Constant.MONTHS_LEN[self.month-1]:
            #     self.day=1
            #     self.month+=1
            # if self.month>12:
            #     self.month=1
            #     self.day=1
            next_obs=self.reset()
            
        else:
            next_obs=self._build_state()
        return current_obs,next_obs,float(reward),finish
    def render(self, current_obs, next_obs, reward, finish):
        print('day={},hour={:2d}, state={}, next_state={}, reward={:.4f}, terminal={}\n'.format(self.day,self.current_time, current_obs, next_obs, reward, finish))
    def _load_year_data(self):
        pv_df=pv_data
        #hourly price data for a year 
        price_df=price_data        # mins electricity consumption data for a year 
        electricity_df=h4_data
        pv_data=pv_df['P_PV_'].apply(lambda x: x.replace(',','.')).to_numpy(dtype=float)
        price=price_df['Price'].apply(lambda x:x.replace(',','.')).to_numpy(dtype=float)
        electricity=electricity_df['Power'].apply(lambda x:x.replace(',','.')).to_numpy(dtype=float)
        # netload=electricity-pv_data
        '''we carefully redesign the magnitude for price and amount of generation as well as demand'''
        for element in pv_data:
            self.data_manager.add_pv_element(element*200)
        for element in price:
            element/=10
            if element<=0.5:
                element=0.5
            self.data_manager.add_price_element(element)
        for i in range(0,electricity.shape[0],60):
            element=electricity[i:i+60]
            self.data_manager.add_electricity_element(sum(element)*300)
if __name__ == '__main__':
    env=ESSEnv()
    env.TRAIN=False
    rewards=[]

    current_obs=env.reset()
    tem_action=[0.1,0.1,0.1,0.1]
    for _ in range (144):
        print(f'current month is {env.month}, current day is {env.day}, current time is {env.current_time}')
        current_obs,next_obs,reward,finish=env.step(tem_action)
        env.render(current_obs,next_obs,reward,finish)
        current_obs=next_obs
        rewards.append(reward)

Rearrange data, so we have daily indices with hourly values in the columns 


In [ ]:
grouped_prices = data.loc[:, ['Price']].groupby(data.index.hour)
daily_prices = pd.DataFrame(index=data.loc[data.index.hour==0].index, columns=range(24), data={h: grouped_prices.get_group(h).values.flatten() for h in range(24)})

In [ ]:
grouped_load = data.loc[:, ['Load']].groupby(data.index.hour)
daily_load = pd.DataFrame(index=data.loc[data.index.hour==0].index, columns=range(24), data={h: grouped_load.get_group(h).values.flatten() for h in range(24)})

Make a feature dataframe with lagged loads and prices. We include the data of the last three days, and the same day last week (d-1, d-2, d-3, d-7)


In [ ]:
lags = [1, 2, 3, 7]

X = pd.concat([daily_prices.shift(l) for l in lags] + [daily_load.shift(l) for l in lags], axis=1).astype(float)
X.columns = [f'P_{h}_{l}' for l in lags for h in range(24)] + [f'L_{h}_{l}' for l in lags for h in range(24)]

Now add the day of the week as a feature. We make a difference for the LASSO (linear model) and the Neural Network (non-linear model). LASSO needs one-hot encoding for these features, while the neural network can deal with a single weekday column having integers representing days.

In [ ]:
X_lasso = X.copy()
for d in range(7):
    X_lasso.loc[X_lasso.index.weekday==d, f'D_{d}']=1
    X_lasso.loc[X_lasso.index.weekday!=d, f'D_{d}']=0

In [ ]:
X_mlp = X.copy()
X_mlp.loc[:, 'DOW'] = X_mlp.index.weekday

In [ ]:
X_lasso.dropna(inplace=True)
X_mlp.dropna(inplace=True)

y = daily_prices.loc[(daily_prices.index.isin(X_lasso.index)) & (daily_prices.index.isin(X_mlp.index))]

Split the data in train-test sets. We use 2019 as test set and the rest as training set. For the lasso we use a different model per hour, and use a single train-test split where we decide the lasso-coefficient on the training data only.

## Define RL algorithms DDPG and auxiliary functions 

### Define the DDPG algorithm


In [ ]:
class Critic(nn.Module):
    def __init__(self, mid_dim, state_dim, action_dim):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(state_dim + action_dim, mid_dim), nn.ReLU(),
                                 nn.Linear(mid_dim, mid_dim), nn.ReLU(),
                                 nn.Linear(mid_dim, mid_dim), nn.Hardswish(),
                                 nn.Linear(mid_dim, 1))

    def forward(self, state, action):
        return self.net(torch.cat((state, action), dim=1))  # q value


class CriticAdv(nn.Module):
    def __init__(self, mid_dim, state_dim, _action_dim):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(state_dim, mid_dim), nn.ReLU(),
                                 nn.Linear(mid_dim, mid_dim), nn.ReLU(),
                                 nn.Linear(mid_dim, mid_dim), nn.Hardswish(),
                                 nn.Linear(mid_dim, 1))

    def forward(self, state):
        return self.net(state)  # advantage value

In [ ]:
class Actor(nn.Module):
    def __init__(self, mid_dim, state_dim, action_dim):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(state_dim, mid_dim), nn.ReLU(),
                                 nn.Linear(mid_dim, mid_dim), nn.ReLU(),
                                 nn.Linear(mid_dim, mid_dim), nn.Hardswish(),
                                 nn.Linear(mid_dim, action_dim))

    def forward(self, state):
        return self.net(state).tanh()  # action.tanh()

    def get_action(self, state, action_std):
        action = self.net(state).tanh()
        noise = (torch.randn_like(action) * action_std).clamp(-0.5, 0.5)
        return (action + noise).clamp(-1.0, 1.0)

In [ ]:
class AgentBase:
    def __init__(self):
        self.state = None
        self.device = None
        self.action_dim = None
        self.if_off_policy = None
        self.explore_noise = None
        self.trajectory_list = None

        self.criterion = torch.nn.SmoothL1Loss()
        self.cri = self.cri_target = self.if_use_cri_target = self.cri_optim = self.ClassCri = None
        self.act = self.act_target = self.if_use_act_target = self.act_optim = self.ClassAct = None

    def init(self, net_dim, state_dim, action_dim, learning_rate=1e-4, _if_per_or_gae=False, gpu_id=0):
        # explict call self.init() for multiprocessing
        self.device = torch.device(f"cuda:{gpu_id}" if (torch.cuda.is_available() and (gpu_id >= 0)) else "cpu")
        self.action_dim = action_dim

        self.cri = self.ClassCri(net_dim, state_dim, action_dim).to(self.device)
        self.act = self.ClassAct(net_dim, state_dim, action_dim).to(self.device) if self.ClassAct else self.cri
        self.cri_target = deepcopy(self.cri) if self.if_use_cri_target else self.cri
        self.act_target = deepcopy(self.act) if self.if_use_act_target else self.act

        self.cri_optim = torch.optim.Adam(self.cri.parameters(), learning_rate)
        self.act_optim = torch.optim.Adam(self.act.parameters(), learning_rate) if self.ClassAct else self.cri
        del self.ClassCri, self.ClassAct

    def select_action(self, state) -> np.ndarray:
        states = torch.as_tensor((state,), dtype=torch.float32, device=self.device)
        action = self.act(states)[0]
        action = (action + torch.randn_like(action) * self.explore_noise).clamp(-1, 1)
        return action.detach().cpu().numpy()
    def explore_env(self, env, target_step):
        trajectory = list()

        state = self.state
        for _ in range(target_step):
            action = self.select_action(state)
            
            state,next_state, reward, done, = env.step(action)

            trajectory.append((state, (reward, done, *action)))
            state = env.reset() if done else next_state
        self.state = state
        return trajectory
    @staticmethod
    def optim_update(optimizer, objective):
        optimizer.zero_grad()
        objective.backward()
        optimizer.step()

    @staticmethod
    def soft_update(target_net, current_net, tau):
        for tar, cur in zip(target_net.parameters(), current_net.parameters()):
            tar.data.copy_(cur.data * tau + tar.data * (1.0 - tau))

    def save_or_load_agent(self, cwd, if_save):
        def load_torch_file(model_or_optim, _path):
            state_dict = torch.load(_path, map_location=lambda storage, loc: storage)
            model_or_optim.load_state_dict(state_dict)

        name_obj_list = [('actor', self.act), ('act_target', self.act_target), ('act_optim', self.act_optim),
                         ('critic', self.cri), ('cri_target', self.cri_target), ('cri_optim', self.cri_optim), ]
        name_obj_list = [(name, obj) for name, obj in name_obj_list if obj is not None]
        if if_save:
            for name, obj in name_obj_list:
                save_path = f"{cwd}/{name}.pth"
                torch.save(obj.state_dict(), save_path)
        else:
            for name, obj in name_obj_list:
                save_path = f"{cwd}/{name}.pth"
                load_torch_file(obj, save_path) if os.path.isfile(save_path) else None

In [ ]:
class AgentDDPG(AgentBase):
    def __init__(self):
        super().__init__()
        self.explore_noise = 0.1  # explore noise of action
        self.if_use_cri_target = self.if_use_act_target = True
        self.ClassCri = Critic
        self.ClassAct = Actor

    def update_net(self, buffer, batch_size, repeat_times, soft_update_tau) -> (float, float):
        buffer.update_now_len()
        obj_critic = obj_actor = None
        for _ in range(int(buffer.now_len / batch_size * repeat_times)):
            obj_critic, state = self.get_obj_critic(buffer, batch_size)#critic loss 
            self.optim_update(self.cri_optim, obj_critic)
            self.soft_update(self.cri_target, self.cri, soft_update_tau)

            action_pg = self.act(state)  # policy gradient
            obj_actor = -self.cri(state, action_pg).mean()# actor loss, makes it bigger
            self.optim_update(self.act_optim, obj_actor)
            self.soft_update(self.act_target, self.act, soft_update_tau)
        return obj_actor.item(), obj_critic.item()

    def get_obj_critic(self, buffer, batch_size) -> (torch.Tensor, torch.Tensor):
        with torch.no_grad():
            reward, mask, action, state, next_s = buffer.sample_batch(batch_size)
            next_q = self.cri_target(next_s, self.act_target(next_s))
            q_label = reward + mask * next_q
        q_value = self.cri(state, action)
        obj_critic = self.criterion(q_value, q_label)
        return obj_critic, state

In [ ]:
def update_buffer(_trajectory):
    ten_state = torch.as_tensor([item[0] for item in _trajectory], dtype=torch.float32)
    ary_other = torch.as_tensor([item[1] for item in _trajectory])
    ary_other[:, 0] = ary_other[:, 0]   # ten_reward
    ary_other[:, 1] = (1.0 - ary_other[:, 1]) * gamma  # ten_mask = (1.0 - ary_done) * gamma

    buffer.extend_buffer(ten_state, ary_other)

    _steps = ten_state.shape[0]
    _r_exp = ary_other[:, 0].mean()  # other = (reward, mask, action)
    return _steps, _r_exp

### Define auxiliary functions 


In [ ]:
def update_buffer(_trajectory):
    ten_state = torch.as_tensor([item[0] for item in _trajectory], dtype=torch.float32)
    ary_other = torch.as_tensor([item[1] for item in _trajectory])
    ary_other[:, 0] = ary_other[:, 0]   # ten_reward
    ary_other[:, 1] = (1.0 - ary_other[:, 1]) * gamma  # ten_mask = (1.0 - ary_done) * gamma

    buffer.extend_buffer(ten_state, ary_other)

    _steps = ten_state.shape[0]
    _r_exp = ary_other[:, 0].mean()  # other = (reward, mask, action)
    return _steps, _r_exp

In [ ]:
class Arguments:
    '''revise here for our own purpose'''
    def __init__(self, agent=None, env=None):
        self.agent = agent  # Deep Reinforcement Learning algorithm
        self.env = env  # the environment for training
        self.plot_shadow_on=False# control do we need to plot all shadow figures
        self.cwd = None  # current work directory. None means set automatically
        self.if_remove = False  # remove the cwd folder? (True, False, None:ask me)
        # self.replace_train_data=True
        self.visible_gpu = '0,1,2,3'  # for example: os.environ['CUDA_VISIBLE_DEVICES'] = '0, 2,'
        self.worker_num = 2  # rollout workers number pre GPU (adjust it to get high GPU usage)
        self.num_threads = 8  # cpu_num for evaluate model, torch.set_num_threads(self.num_threads)

        '''Arguments for training'''
        self.num_episode=2000
        self.gamma = 0.995  # discount factor of future rewards
        # self.reward_scale = 1  # an approximate target reward usually be closed to 256
        self.learning_rate = 2 ** -14  # 2 ** -14 ~= 6e-5
        self.soft_update_tau = 2 ** -8  # 2 ** -8 ~= 5e-3

        self.net_dim = 256  # the network width 256
        self.batch_size = 4096  # num of transitions sampled from replay buffer.
        self.repeat_times = 2 ** 5  # repeatedly update network to keep critic's loss small
        self.target_step = 4096 # collect target_step experiences , then update network, 1024
        self.max_memo = 500000  # capacity of replay buffer
        self.if_per_or_gae = False  # PER for off-policy sparse reward: Prioritized Experience Replay.

        '''Arguments for evaluate'''
        # self.eval_gap = 2 ** 6  # evaluate the agent per eval_gap seconds
        # self.eval_times = 2  # number of times that get episode return in first
        self.random_seed = 0  # initialize random seed in self.init_before_training()
        self.random_seed_list=[1234,2234,3234,4234,5234]
        '''Arguments for save and plot issues'''
        self.train=True
        self.save_network=True
        self.test_network=True
        self.save_test_data=True
        self.compare_with_pyomo=True
        self.plot_on=True 

    def init_before_training(self, if_main):
        if self.cwd is None:
            agent_name = self.agent.__class__.__name__
            self.cwd = f'./{agent_name}'

        if if_main:
            import shutil  # remove history according to bool(if_remove)
            if self.if_remove is None:
                self.if_remove = bool(input(f"| PRESS 'y' to REMOVE: {self.cwd}? ") == 'y')
            elif self.if_remove:
                shutil.rmtree(self.cwd, ignore_errors=True)
                print(f"| Remove cwd: {self.cwd}")
            os.makedirs(self.cwd, exist_ok=True)

        np.random.seed(self.random_seed)
        torch.manual_seed(self.random_seed)
        torch.set_num_threads(self.num_threads)
        torch.set_default_dtype(torch.float32)

        os.environ['CUDA_VISIBLE_DEVICES'] = str(self.visible_gpu)

In [ ]:
class ReplayBuffer:
    def __init__(self, max_len, state_dim, action_dim, gpu_id=0):
        self.now_len = 0
        self.next_idx = 0
        self.if_full = False
        self.max_len = max_len
        self.data_type = torch.float32
        self.action_dim = action_dim
        self.device = torch.device(f"cuda:{gpu_id}" if (torch.cuda.is_available() and (gpu_id >= 0)) else "cpu")

        other_dim = 1 + 1 + self.action_dim
        self.buf_other = torch.empty(size=(max_len, other_dim), dtype=self.data_type, device=self.device)

        if isinstance(state_dim, int):  # state is pixel
            self.buf_state = torch.empty((max_len, state_dim), dtype=torch.float32, device=self.device)
        elif isinstance(state_dim, tuple):
            self.buf_state = torch.empty((max_len, *state_dim), dtype=torch.uint8, device=self.device)
        else:
            raise ValueError('state_dim')

    def extend_buffer(self, state, other):  # CPU array to CPU array
        size = len(other)
        next_idx = self.next_idx + size

        if next_idx > self.max_len:
            self.buf_state[self.next_idx:self.max_len] = state[:self.max_len - self.next_idx]
            self.buf_other[self.next_idx:self.max_len] = other[:self.max_len - self.next_idx]
            self.if_full = True

            next_idx = next_idx - self.max_len
            self.buf_state[0:next_idx] = state[-next_idx:]
            self.buf_other[0:next_idx] = other[-next_idx:]
        else:
            self.buf_state[self.next_idx:next_idx] = state
            self.buf_other[self.next_idx:next_idx] = other
        self.next_idx = next_idx

    def sample_batch(self, batch_size) -> tuple:
        indices = rd.randint(self.now_len - 1, size=batch_size)
        r_m_a = self.buf_other[indices]
        return (r_m_a[:, 0:1],
                r_m_a[:, 1:2],
                r_m_a[:, 2:],
                self.buf_state[indices],
                self.buf_state[indices + 1])

    def update_now_len(self):
        self.now_len = self.max_len if self.if_full else self.next_idx

In [ ]:
def get_episode_return(env, act, device):
    episode_return = 0.0  # sum of rewards in an episode
    episode_unbalance=0.0
    state = env.reset()
    for i in range(24):
        s_tensor = torch.as_tensor((state,), device=device)
        a_tensor = act(s_tensor)
        action = a_tensor.detach().cpu().numpy()[0]  # not need detach(), because with torch.no_grad() outside
        state, next_state, reward, done,= env.step(action)
        state=next_state
        episode_return += reward
        episode_unbalance+=env.real_unbalance
        if done:
            break
    return episode_return,episode_unbalance

## Train and evaluate by DDPG algorithms 

[Least absolute shrinkage and selection operator (LASSO)](https://www.jstor.org/stable/2346178#metadata_info_tab_contents), is a regression analysis method that performs both variable selection and regularization in order to enhance the prediction accuracy and interpretability of the resulting statistical model.

Lasso was originally formulated for linear regression models. This simple case reveals a substantial amount about the estimator. These include its relationship to ridge regression and best subset selection and the connections between lasso coefficient estimates and so-called soft thresholding. It also reveals that (like standard linear regression) the coefficient estimates do not need to be unique if covariates are collinear.

In this part, LASSO is implemented to train the price forecast model.

In [ ]:
args=Arguments()
    '''here record real unbalance'''
    reward_record={'episode':[],'steps':[],'mean_episode_reward':[],'unbalance':[]}
    loss_record={'episode':[],'steps':[],'critic_loss':[],'actor_loss':[],'entropy_loss':[]}
    args.visible_gpu = '2'
    for seed in args.random_seed_list:
        args.random_seed=seed
        #set different seed 
        args.agent=AgentDDPG()
        agent_name=f'{args.agent.__class__.__name__}'
        args.agent.cri_target=True
        args.env=ESSEnv()
        # creat lists of lists/or creat a long list? 
        
        args.init_before_training(if_main=True)
        '''init agent and environment'''
        agent=args.agent
        env=args.env
        agent.init(args.net_dim,env.state_space.shape[0],env.action_space.shape[0],args.learning_rate,args.if_per_or_gae)
        '''init replay buffer'''
        buffer = ReplayBuffer(max_len=args.max_memo, state_dim=env.state_space.shape[0],
                            action_dim= env.action_space.shape[0])    
        '''start training'''
        cwd=args.cwd
        gamma=args.gamma
        batch_size=args.batch_size# how much data should be used to update net
        target_step=args.target_step#how manysteps of one episode should stop
        # reward_scale=args.reward_scale# here we use it as 1# we dont need this in our model
        repeat_times=args.repeat_times# how many times should update for one batch size data 
        # if_allow_break = args.if_allow_break
        soft_update_tau = args.soft_update_tau
        # get the first experience from 
        agent.state=env.reset()
        # trajectory=agent.explore_env(env,target_step)
        # update_buffer(trajectory)
        '''collect data and train and update network'''
        num_episode=args.num_episode

        ##
        # args.train=False
        # args.save_network=False
        # args.test_network=False
        # args.save_test_data=False
        # args.compare_with_pyomo=False
        #
        if args.train:
            collect_data=True
            while collect_data:
                print(f'buffer:{buffer.now_len}')
                with torch.no_grad():
                    trajectory=agent.explore_env(env,target_step)
                    steps,r_exp=update_buffer(trajectory)
                    buffer.update_now_len()
                if buffer.now_len>=10000:
                    collect_data=False
            for i_episode in range(num_episode):
                critic_loss,actor_loss=agent.update_net(buffer,batch_size,repeat_times,soft_update_tau)
                loss_record['critic_loss'].append(critic_loss)
                loss_record['actor_loss'].append(actor_loss)
                with torch.no_grad():
                    episode_reward,episode_unbalance=get_episode_return(env,agent.act,agent.device)
                    reward_record['mean_episode_reward'].append(episode_reward)
                    reward_record['unbalance'].append(episode_unbalance)
                print(f'curren epsiode is {i_episode}, reward:{episode_reward},unbalance:{episode_unbalance},buffer_length: {buffer.now_len}')
                if i_episode % 10==0:
                    # target_step
                    with torch.no_grad():
                        trajectory=agent.explore_env(env,target_step)
                        steps,r_exp=update_buffer(trajectory)
    loss_record_path=f'{args.cwd}/loss_data.pkl'
    reward_record_path=f'{args.cwd}/reward_data.pkl'
    # current only store last seed corresponded actor 
    act_save_path=f'{args.cwd}/actor.pth'
    # args.replace_train_data = False

    with open (loss_record_path,'wb') as tf:
        pickle.dump(loss_record,tf)
    with open (reward_record_path,'wb') as tf:
        pickle.dump(reward_record,tf)
    print('training data have been saved')
    if args.save_network:
        torch.save(agent.act.state_dict(),act_save_path)
        print('actor parameters have been saved')